In [1]:
'''Random search and grid search are limited because they blindly explore the hyperparameter space—grid search is exhaustive and expensive, while random search samples without learning from past trials. ⚡ Optuna, on the other hand, uses Bayesian optimization + pruning strategies, making it adaptive: it learns from previous results to focus on the most promising regions, reducing computation time and improving performance.'''

'Random search and grid search are limited because they blindly explore the hyperparameter space—grid search is exhaustive and expensive, while random search samples without learning from past trials. ⚡ Optuna, on the other hand, uses Bayesian optimization + pruning strategies, making it adaptive: it learns from previous results to focus on the most promising regions, reducing computation time and improving performance.'

In [4]:
##optuna hyperparameter tuning(bayseian search)
'''accuracy is function of hyperparameters
it establish the relationship between accuracy or loss between the hyperparameter
and make curve with that and find the min loss or maxm accuracy in the graph
it learns from the past values to lessen down the computation'''


'accuracy is function of hyperparameters\nit establish the relationship between accuracy or loss between the hyperparameter\nand make curve with that and find the min loss or maxm accuracy in the graph\nit learns from the past values to lessen down the computation'

In [5]:
##key terms for optuna
'''
1)Study
2)Trial
3)Trials parameters
4)Objective function
5)Sampler="TPE"
'''

'\n1)Study\n2)Trial\n3)Trials parameters\n4)Objective function\n5)Sampler="TPE"\n'

In [6]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 21.0 MB/s eta 0:00:00


In [7]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [9]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [11]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-10-02 12:43:24,585] A new study created in memory with name: no-name-326ec3ee-c79f-4f21-81b3-c95395c53afc
[I 2025-10-02 12:43:26,385] Trial 0 finished with value: 0.7783985102420856 and parameters: {'n_estimators': 122, 'max_depth': 19}. Best is trial 0 with value: 0.7783985102420856.
[I 2025-10-02 12:43:27,235] Trial 1 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 137, 'max_depth': 9}. Best is trial 0 with value: 0.7783985102420856.
[I 2025-10-02 12:43:28,166] Trial 2 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 156, 'max_depth': 6}. Best is trial 0 with value: 0.7783985102420856.
[I 2025-10-02 12:43:28,816] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 102, 'max_depth': 8}. Best is trial 0 with value: 0.7783985102420856.
[I 2025-10-02 12:43:29,600] Trial 4 finished with value: 0.7783985102420857 and parameters: {'n_estimators': 126, 'max_depth': 7}. Best is trial 4 with value: 0.77839851

In [12]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 120, 'max_depth': 7}


In [13]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.75


In [15]:
##Samplers in optuna(Random Sampler)

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [17]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-10-02 12:49:25,179] A new study created in memory with name: no-name-14ce7127-c190-4ed1-967c-7a5cc98b2f13
[I 2025-10-02 12:49:26,241] Trial 0 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 106, 'max_depth': 6}. Best is trial 0 with value: 0.7653631284916201.
[I 2025-10-02 12:49:26,776] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 74, 'max_depth': 16}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-10-02 12:49:27,416] Trial 2 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 99, 'max_depth': 17}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-10-02 12:49:27,849] Trial 3 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 69, 'max_depth': 10}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-10-02 12:49:29,060] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 190, 'max_depth': 19}. Best is trial 1 with value: 0.77467411

In [18]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 50, 'max_depth': 7}


In [19]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.73


In [20]:
##ANOther Sampler grid search cv but you have to define search space first

In [21]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [22]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-10-02 12:51:36,719] A new study created in memory with name: no-name-cecc73d9-6e98-4eee-a474-6f09fdeca9ed
[I 2025-10-02 12:51:38,348] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-02 12:51:40,928] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-02 12:51:41,586] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-10-02 12:51:42,733] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-10-02 12:51:44,215] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [23]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [24]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [25]:
## Optuna Visualizations

In [26]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [27]:
# 1. Optimization History
plot_optimization_history(study).show()

In [28]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [29]:
# 3. Slice Plot
plot_slice(study).show()

In [30]:
# 4. Contour Plot
plot_contour(study).show()

In [31]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [33]:
## Optimizing Multiple ML Models

In [34]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [35]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [36]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-10-02 13:00:33,060] A new study created in memory with name: no-name-752029cf-396b-4258-9a4e-f84b066bd1e8
[I 2025-10-02 13:00:33,106] Trial 0 finished with value: 0.7765363128491619 and parameters: {'classifier': 'SVM', 'C': 0.19736177427216967, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.7765363128491619.
[I 2025-10-02 13:00:35,709] Trial 1 finished with value: 0.7560521415270017 and parameters: {'classifier': 'RandomForest', 'n_estimators': 294, 'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 9, 'bootstrap': True}. Best is trial 0 with value: 0.7765363128491619.
[I 2025-10-02 13:00:35,744] Trial 2 finished with value: 0.7839851024208566 and parameters: {'classifier': 'SVM', 'C': 1.214179847879685, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 2 with value: 0.7839851024208566.
[I 2025-10-02 13:00:36,275] Trial 3 finished with value: 0.7672253258845437 and parameters: {'classifier': 'RandomForest', 'n_estimators': 92, 'max_depth': 5, '

In [37]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11079920654588875, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [38]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.776536,2025-10-02 13:00:33.062533,2025-10-02 13:00:33.106347,0 days 00:00:00.043814,0.197362,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.756052,2025-10-02 13:00:33.107589,2025-10-02 13:00:35.709567,0 days 00:00:02.601978,NaN,True,RandomForest,NaN,NaN,NaN,8.0,9.0,9.0,294.0,COMPLETE
2,2,0.783985,2025-10-02 13:00:35.710618,2025-10-02 13:00:35.744916,0 days 00:00:00.034298,1.214180,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.767225,2025-10-02 13:00:35.746108,2025-10-02 13:00:36.275482,0 days 00:00:00.529374,NaN,True,RandomForest,NaN,NaN,NaN,5.0,10.0,8.0,92.0,COMPLETE
4,4,0.729981,2025-10-02 13:00:36.276522,2025-10-02 13:00:42.452920,0 days 00:00:06.176398,NaN,NaN,GradientBoosting,NaN,NaN,0.016589,10.0,3.0,3.0,266.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.715084,2025-10-02 13:01:15.494935,2025-10-02 13:01:15.547316,0 days 00:00:00.052381,0.154788,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2025-10-02 13:01:15.548283,2025-10-02 13:01:15.598737,0 days 00:00:00.050454,0.207206,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.789572,2025-10-02 13:01:15.601040,2025-10-02 13:01:15.648591,0 days 00:00:00.047551,0.128908,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-10-02 13:01:15.649491,2025-10-02 13:01:15.698093,0 days 00:00:00.048602,0.328424,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [39]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
RandomForest,12
GradientBoosting,9


In [40]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.743637
RandomForest,0.763191
SVM,0.774721


In [41]:
# 1. Optimization History
plot_optimization_history(study).show()